<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

# Exercise 6.3 Answers


(a) Set up

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN, LSTM, GRU, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences

(b) import phone data

In [0]:
# coding
PHONES={ "sil":0, "b":1, "d":2, "g":3, "p":4, "t":5, "k":6, "dZ":7, "tS":8, "m":9, "n":10, "N":11, "v":12, "D":13, "z":14, "Z":15, "f":16, "T":17, "s":18, "S":19, "h":20, "r":21, "w":22, "l":23, "j":24, "i:":25, "I":26, "e":27, "{":28, "V":29, "A:":30, "O:":31, "U":32, "u:":33, "3:":34, "@":35, "aI":36, "eI":37, "OI":38, "aU":39, "@U":40 }

def prepare_data_single(filename):
  df=pd.read_csv(filename)
  nvec=len(df)
  feats=df.iloc[:,2:].to_numpy();
  labels=np.array(list(map(PHONES.get, df.LABEL)))
  p = np.random.permutation(nvec)
  return feats[p,:],labels[p]

Xtest, ytest = prepare_data_single("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-test.csv")
Xtrain, ytrain = prepare_data_single("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-train.csv")
Xval, yval = prepare_data_single("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-valid.csv")

print(Xtrain.shape,ytrain.shape)
print(Xval.shape,yval.shape)
print(Xtest.shape,ytest.shape)

print(Xtrain[:10,:])
print(ytrain[:10])

(c) build a simple model

In [0]:
isize=13
osize=41

model = Sequential()
model.add(Dense(40,activation='tanh',input_shape=(isize,)))
model.add(Dense(40,activation='tanh'))
model.add(Dense(osize, activation='softmax'));
#
# compile the network
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

(d) train simple model

In [0]:
# train the model
history=model.fit(Xtrain,ytrain, batch_size=256, validation_data=(Xval,yval), epochs=25)
print(history.history)

(e) import data for sequence model

In [0]:
def prepare_data_sequence(filename,seqlen):
  df=pd.read_csv(filename)
  nvec=len(df)
  nfeat=len(df.columns)-2
  nseq=nvec//seqlen
  feats=df.iloc[:(nseq*seqlen),2:].to_numpy();
  feats=np.reshape(feats,(nseq,seqlen,nfeat))
  labels=np.array(list(map(PHONES.get, df.LABEL)))
  labels=np.reshape(labels[:(nseq*seqlen)],(nseq,seqlen))
  p = np.random.permutation(nseq)
  return feats[p,:,:],labels[p,:]

Xtest, ytest = prepare_data_sequence("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-test.csv",200)
Xtrain, ytrain = prepare_data_sequence("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-train.csv",200)
Xval, yval = prepare_data_sequence("https://www.phon.ucl.ac.uk/courses/pals0039/data/phone-valid.csv",200)

print(Xtrain.shape,ytrain.shape)
print(Xval.shape,yval.shape)
print(Xtest.shape,ytest.shape)


(f) build a sequence model

In [0]:
seqlen=200
isize=13
osize=41

model = Sequential()
model.add(Bidirectional(LSTM(32,return_sequences=True),input_shape=(seqlen,isize),merge_mode='ave'))
model.add(Bidirectional(LSTM(32,return_sequences=True),merge_mode='ave'))
model.add(TimeDistributed(Dense(osize, activation='softmax')))
#
# compile the network
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

(g) train sequence model

In [0]:
# train the model
history=model.fit(Xtrain,ytrain, batch_size=16, validation_data=(Xval,yval), epochs=25)
print(history.history)

(h) analyse performance by phone

In [0]:
ypred=model.predict(Xtest)
print(ypred.shape)
ypred=np.reshape(ypred,(ypred.shape[0]*ypred.shape[1],ypred.shape[2]))
print(ypred.shape)
ypred=np.argmax(ypred,axis=1)
print(ypred.shape)
ytrue=np.reshape(ytest,(ytest.shape[0]*ytest.shape[1],))
print(ytrue.shape)

def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):
    plt.figure(figsize=(10,10))
    plt.matshow(df_confusion, cmap=cmap,fignum=1) # imshow
    #plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    plt.show()

df_confusion = pd.crosstab(ytrue, ypred,rownames=['True'],colnames=['Predicted'])
plot_confusion_matrix(df_confusion)

from nltk import ConfusionMatrix

id2phn={}
for k,v in PHONES.items():
  id2phn[v]=k

phone_true=[]
phone_pred=[]
for i in range(len(ytrue)):
  phone_true.append(id2phn[ytrue[i]])
  phone_pred.append(id2phn[ypred[i]])

print(len(phone_true),len(phone_pred))

print(ConfusionMatrix(phone_true,phone_pred))

df_confusion = pd.crosstab(pd.Series(phone_true), pd.Series(phone_pred),rownames=['True'],colnames=['Predicted'])
plot_confusion_matrix(df_confusion)

from sklearn.metrics import classification_report, confusion_matrix

labellist=PHONES.keys()
print(labellist)
print('Confusion Matrix')
np.set_printoptions(threshold=np.inf)
print(confusion_matrix(ytrue, ypred))
print('Classification Report')
print(classification_report(ytrue, ypred, labels=range(41), target_names=labellist))	

(i) plot phone posteriors

In [0]:
ypred=model.predict(Xtest)
seq=ypred[0]
lab=ytest[0]
phn=[id2phn[lab[0]]]
for l in lab:
  if id2phn[l] != phn[-1]:
    phn.append(id2phn[l])
#print(seq)
#print(lab)
#print(phn)

plt.figure(figsize=(15,10))
plt.imshow(seq.T,origin='lower',cmap='binary',aspect='auto')
ax = plt.gca()
ax.set_yticks(range(41))
ax.set_yticklabels(labellist)
plt.title(" ".join(phn))
plt.show()